<a href="https://colab.research.google.com/github/bragarods/rainfall_forecast/blob/master/notebooks/colab_prophet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from io import StringIO
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn.metrics as skm
import warnings

plt.rcParams['figure.figsize'] = (13, 7)

In [ ]:
!pip3 install --upgrade pystan
!pip3 install --upgrade fbprophet

## Import monthly data

In [28]:
# google drive shareable file link

orig_url = 'https://drive.google.com/file/d/1WQ9NARYCNZxWlQUAf4Zn5TxJcnOGjEn3/view?usp=sharing'

# get file id

file_id = orig_url.split('/')[-2]

# create download url

dwn_url='https://drive.google.com/uc?export=download&id=' + file_id

# get raw text inside url

url = requests.get(dwn_url).text

# create a buffer

csv_raw = StringIO(url)

# read from buffer

df = pd.read_csv(csv_raw, index_col=0)

df.reset_index(drop=True, inplace=True)

# create sinop and csinop series

df['date'] = pd.to_datetime(df['date'], format=('%Y-%m-%d')) 

df.set_index('date', inplace=True)

# cut train period

train = df[(df.index >= '2000-01-01') & (df.index <= '2019-12-01')]

test = df[df.index >= '2020-01-01']

## Flag first full year of data

In [ ]:
df['chuva_max_12m'] = df.groupby('cd_estacao')['chuva'].rolling(12).max().reset_index(0,drop=True)

df['flag_chuva_max_12m'] = np.where((~df['chuva_max_12m'].isna()) & (df['chuva_max_12m'] >= 1),
                               1,
                               0)

min_year = df[df['flag_chuva_max_12m']==1].groupby('cd_estacao').agg(min_year=('date','min'))['min_year'].dt.year+1

df = pd.merge(df,min_year,how='left',left_on=df.cd_estacao,right_on=min_year.index)

df.drop(columns='key_0', inplace=True)

## Exploratory

In [5]:
strain = train[(train['dc_nome']=='sinop') & (train.index>='2009-01-01') & (train.index<'2020-01-01')].chuva
stest = test[(test['dc_nome']=='sinop') & (test.index>='2020-01-01')].chuva

In [ ]:
# time series decomposition

res = sm.tsa.seasonal_decompose(strain, period=12, extrapolate_trend='freq')
resplot = res.plot()

In [ ]:
def tsplot(y, lags=None, figsize=(10, 8), style='bmh'):
    if not isinstance(y, pd.Series):
        y = pd.Series(y)
    with plt.style.context(style):    
        fig = plt.figure(figsize=figsize)
        #mpl.rcParams['font.family'] = 'Ubuntu Mono'
        layout = (3, 2)
        ts_ax = plt.subplot2grid(layout, (0, 0), colspan=2)
        acf_ax = plt.subplot2grid(layout, (1, 0))
        pacf_ax = plt.subplot2grid(layout, (1, 1))
        qq_ax = plt.subplot2grid(layout, (2, 0))
        pp_ax = plt.subplot2grid(layout, (2, 1))
        
        y.plot(ax=ts_ax)
        ts_ax.set_title('Time Series Analysis Plots')
        smt.graphics.plot_acf(y, lags=lags, ax=acf_ax, alpha=0.5)
        smt.graphics.plot_pacf(y, lags=lags, ax=pacf_ax, alpha=0.5)
        sm.qqplot(y, line='s', ax=qq_ax)
        qq_ax.set_title('QQ Plot')        
        scs.probplot(y, sparams=(y.mean(), y.std()), plot=pp_ax)

        plt.tight_layout()

In [ ]:
tsplot(strain)

## Sarimax

In [4]:
from fbprophet import Prophet

In [9]:
# preparing data for Prophet

strain = strain.reset_index().rename(columns={'date':'ds','chuva':'y'})

In [11]:
m = Prophet()
m.fit(strain)

INFO:numexpr.utils:NumExpr defaulting to 2 threads.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [16]:
future = m.make_future_dataframe(periods=12, freq='MS')
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(12)

,ds,yhat,yhat_lower,yhat_upper
132,2020-01-01,253.680849,130.425408,361.950700
133,2020-02-01,195.160244,73.788728,312.934060
134,2020-03-01,209.085492,85.181681,319.959668
135,2020-04-01,80.181761,-29.254333,191.205072
136,2020-05-01,38.858865,-68.527538,159.568824
137,2020-06-01,28.074353,-85.843613,138.552135
138,2020-07-01,19.067141,-93.056841,134.659565
139,2020-08-01,31.459883,-83.281497,143.077184
140,2020-09-01,60.385226,-52.592669,176.185963
141,2020-10-01,166.228941,51.605974,279.823244


In [19]:
from fbprophet.plot import plot_plotly, plot_components_plotly

plot_plotly(m, forecast)

In [20]:
# Python
plot_components_plotly(m, forecast)

In [45]:
from sklearn.metrics import mean_absolute_error

mean_absolute_error(df[(df.dc_nome=='sinop') & (df.index >= '2009-01-01')].chuva, pd.Series(forecast.yhat[:-1]).clip(0))

63.326747462030504